In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/first-quora-dataset/q_quora.csv


In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


file_path = "/kaggle/input/first-quora-dataset/q_quora.csv"


df = pd.read_csv(file_path, low_memory=False)



print(df.head())
print(df.info())


   id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? How can I solve...   
4   4     9    10  Which one dissolve in water quikly sugar, salt...   

                                           question2 is_duplicate Unnamed: 6  \
0  What is the step by step guide to invest in sh...            0        NaN   
1  What would happen if the Indian government sto...            0        NaN   
2  How can Internet speed be increased by hacking...            0        NaN   
3  Find the remainder when [math]23^{24}[/math] i...            0        NaN   
4            Which fish would survive in salt water?            0        NaN   

  Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10 Unnamed: 11  Unnamed: 12  
0        NaN

In [3]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from multiprocessing import Pool, cpu_count
import pandas as pd


stop_words = set(stopwords.words("english"))
k = WordNetLemmatizer()
pattern = re.compile(f"[{re.escape(string.punctuation)}]")

def textpreprocessing(text):
    if not isinstance(text, str):
        return ""
    
    text = pattern.sub("", text.lower())
    n = [token for token in text.split() if token not in stop_words]
    text = " ".join(n)
    
    u = [k.lemmatize(token) for token in text.split()]
    text = " ".join(u)
    return text


def apply_series(series):
    return series.apply(textpreprocessing)


def parallel_apply(series):
    n_cores = cpu_count()
    chunk_size = int(len(series) / n_cores) + 1
    chunks = [series[i:i + chunk_size] for i in range(0, len(series), chunk_size)]
    
    with Pool(n_cores) as pool:
        results = pool.map(apply_series, chunks)
    
    return pd.concat(results)


df['question1'] = parallel_apply(df['question1'])
df['question2'] = parallel_apply(df['question2'])


In [4]:
sentences1=[q.split() for q in df['question1']]
sentences2=[q.split() for q in df['question2']]

In [5]:

from gensim.models import Word2Vec
model1=Word2Vec(sentences1,vector_size=200,sg=1,min_count=1,workers=4)
model2=Word2Vec(sentences2,vector_size=200,sg=1,min_count=1,workers=4)

In [6]:
def clean_text(text):
    text = text.lower()                
    text = ''.join(ch for ch in text if ch.isalnum() or ch.isspace()) 
    return text


df['question1'] = df['question1'].astype(str).apply(clean_text)
df['question2'] = df['question2'].astype(str).apply(clean_text)


In [7]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab={}
i=1
for w in model1.wv.index_to_key:
    vocab[w]=i
    i+=1
vocab_size=len(vocab)+1

2025-10-06 13:22:42.267827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759756962.489026      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759756962.553616      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
vocab_size


75688

In [9]:
def text_to_sequences(sentence, vocab):
    seq = []
    for w in sentence.split():
        if w in vocab:
            seq.append(vocab[w])   
    return seq

seq1 = []
for q in df['question1']:
    seq1.append(text_to_sequences(q, vocab))

seq2 = []
for q in df['question2']:
    seq2.append(text_to_sequences(q, vocab))
x1=pad_sequences(seq1,maxlen=30,padding='post')
x2=pad_sequences(seq2,maxlen=30,padding='post')


In [10]:
x1


array([[ 560,  560, 2074, ...,    0,    0,    0],
       [ 232, 9764, 9764, ...,    0,    0,    0],
       [ 133,  357,  254, ...,    0,    0,    0],
       ...,
       [   9, 1709,    0, ...,    0,    0,    0],
       [6581, 2868,  199, ...,    0,    0,    0],
       [   4,   82, 3430, ...,    0,    0,    0]], dtype=int32)

In [11]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate

embedding_dim = 200
max_len = 30


embedding_matrix = np.zeros((len(vocab) + 1, embedding_dim))

for word, i in vocab.items():
    if word in model1.wv:
        embedding_matrix[i] = model1.wv[word]  


embedding_layer = Embedding(
    input_dim=len(vocab) + 1,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    trainable=False
)


2025-10-06 13:23:03.242374: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [12]:

df['is_duplicate'] = pd.to_numeric(df['is_duplicate'], errors='coerce')


df = df[df['is_duplicate'].notna()]
df['is_duplicate'] = df['is_duplicate'].astype(int)


def text_to_sequences(sentence, vocab):
    seq = []
    for w in sentence.split():
        if w in vocab:
            seq.append(vocab[w])
    return seq

seq1 = [text_to_sequences(q, vocab) for q in df['question1']]
seq2 = [text_to_sequences(q, vocab) for q in df['question2']]


x1 = pad_sequences(seq1, maxlen=30, padding='post')
x2 = pad_sequences(seq2, maxlen=30, padding='post')


y = df['is_duplicate'].values




In [13]:
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Dropout, Lambda
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [14]:
x1_train, x1_test, x2_train, x2_test, y_train, y_test = train_test_split(
    x1, x2, y, test_size=0.2, random_state=42
)
q1_in = Input(shape=(30,))
q2_in = Input(shape=(30,))


q1_embed = embedding_layer(q1_in)
q2_embed = embedding_layer(q2_in)


shared_lstm = Bidirectional(LSTM(64, return_sequences=False))

q1_vec = shared_lstm(q1_embed)
q2_vec = shared_lstm(q2_embed)


diff = Lambda(lambda x: abs(x[0] - x[1]))([q1_vec, q2_vec])
merged = Dense(128, activation='relu',kernel_regularizer='l2')(diff)
merged = Dropout(0.5)(merged)


out = Dense(1, activation='sigmoid')(merged)

model = Model(inputs=[q1_in, q2_in], outputs=out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit([x1_train, x2_train], y_train,
          validation_data=([x1_test, x2_test], y_test),
          epochs=20, batch_size=512, callbacks=[es],
          class_weight={0:1, 1:2})



loss, acc = model.evaluate([x1_test, x2_test], y_test)
print("Test Accuracy:", acc)

Epoch 1/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 206s 312ms/step - accuracy: 0.6664 - loss: 1.0908 - val_accuracy: 0.7466 - val_loss: 0.5084
Epoch 2/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 194s 307ms/step - accuracy: 0.7549 - loss: 0.6478 - val_accuracy: 0.7665 - val_loss: 0.4809
Epoch 3/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 194s 306ms/step - accuracy: 0.7760 - loss: 0.6085 - val_accuracy: 0.7650 - val_loss: 0.4878
Epoch 4/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 196s 310ms/step - accuracy: 0.7894 - loss: 0.5826 - val_accuracy: 0.7933 - val_loss: 0.4438
Epoch 5/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 200s 308ms/step - accuracy: 0.8019 - loss: 0.5617 - val_accuracy: 0.7909 - val_loss: 0.4507
Epoch 6/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 195s 309ms/step - accuracy: 0.8116 - loss: 0.5393 - val_accuracy: 0.7920 - val_loss: 0.4498
Epoch 7/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 195s 308ms/step - accuracy: 0.8178 - loss: 0.5240 - val_accuracy: 0.8102 - val_loss: 0.4208
Epoch 8/20
632/632 ━━━━━━━━━━━━━━━━━━━━ 197s 312ms/step - accuracy: 0.8273 -